In [1]:
import pandas as pd
import numpy as np
pd.set_option('mode.chained_assignment', None)

###  Carga de datos

In [3]:
auctions = pd.read_csv("auctions.csv", dtype={"ref_type_id": np.uint8, "source_id": np.uint8})
auctions["date"] =  pd.to_datetime(auctions["date"], errors = "coerce")

In [153]:
clicks = pd.read_csv('clicks.csv', low_memory=False)
clicks["created"] =  pd.to_datetime(clicks["created"], errors = "coerce")

In [ ]:
events = pd.read_csv('events.csv', low_memory=False)
events["date"] =  pd.to_datetime(events["date"], errors = "coerce")

In [10]:
installs = pd.read_csv('installs.csv', low_memory = False)
installs["created"] =  pd.to_datetime(installs["created"], errors = "coerce") 

### Partición de los datos por ventanas

In [56]:
#la fecha final debe ser las 00hs del día siguiente a la fecha final de la ventana
def create_window(df, initial_date, final_date, date_feature_name):
    initial = pd.to_datetime(initial_date)
    final = pd.to_datetime(final_date)
    return df.loc[ (df[date_feature_name] >= initial) \
                  & (df[date_feature_name] < final) ]

In [57]:
auctions_18_20 =  create_window(auctions, "2019-04-18", "2019-04-21", "date")
auctions_19_21 =  create_window(auctions, "2019-04-19", "2019-04-22", "date")
auctions_20_22 =  create_window(auctions, "2019-04-20", "2019-04-23", "date")
auctions_21_23 =  create_window(auctions, "2019-04-21", "2019-04-24", "date")
auctions_22_24 =  create_window(auctions, "2019-04-22", "2019-04-25", "date")

In [58]:
events_18_20 =  create_window(events, "2019-04-18", "2019-04-21", "date")
events_19_21 =  create_window(events, "2019-04-19", "2019-04-22", "date")
events_20_22 =  create_window(events, "2019-04-20", "2019-04-23", "date")
events_21_23 =  create_window(events, "2019-04-21", "2019-04-24", "date")
events_22_24 =  create_window(events, "2019-04-22", "2019-04-25", "date")

NameError: name 'events' is not defined

In [154]:
clicks_18_20 =  create_window(clicks, "2019-04-18", "2019-04-21", "created")
clicks_19_21 =  create_window(clicks, "2019-04-19", "2019-04-22", "created")
clicks_20_22 =  create_window(clicks, "2019-04-20", "2019-04-23", "created")
clicks_21_23 =  create_window(clicks, "2019-04-21", "2019-04-24", "created")
clicks_22_24 =  create_window(clicks, "2019-04-22", "2019-04-25", "created")

TypeError: Cannot compare tz-naive and tz-aware datetime-like objects

In [60]:
installs_18_20 =  create_window(installs, "2019-04-18", "2019-04-21", "created")
installs_19_21 =  create_window(installs, "2019-04-19", "2019-04-22", "created")
installs_20_22 =  create_window(installs, "2019-04-20", "2019-04-23", "created")
installs_21_23 =  create_window(installs, "2019-04-21", "2019-04-24", "created")
installs_22_24 =  create_window(installs, "2019-04-22", "2019-04-25", "created")

### Inicialización de df de features

In [61]:
#le pasamos a esta funcion el dataframe de la ventana a la cual deseemos agregarle features posteriormente
def initialize_trainning_df(df, index_name):
    trainning_df = pd.DataFrame(df[index_name].unique())
    trainning_df.columns = ['ref_hash']
    trainning_df = trainning_df.set_index('ref_hash')
    return trainning_df

In [62]:
example = initialize_trainning_df(installs_18_20, "ref_hash")
example.head(3)

""
ref_hash
5230323462636548010
5097163995161606833
6328027616411983332


> # Sección para añadir features

> *¡Atención! Antes de usar la función add_feature, setear como indice el id de los dispositivos en el df que contiene el feature que se desea agregar*

In [63]:
def charge_df(csv_name, dict_dtypes = None):
    return pd.read_csv(csv_name, dict_dtypes)

In [64]:
def add_feature(df_features, df_to_join, feature_name, fill = 0):
    df_features = df_features.merge(df_to_join[[feature_name]], how = 'left', left_index = True, right_index = True).\
    fillna(fill)
    return df_features

Para no tener que volver a computar las operaciones, una vez agregados los features podemos guardar el estado final del df de features

In [65]:
def save_changes(df, file_name):
    df.to_csv(file_name, index = True)

# **Auctions**

In [66]:
auc_18_20_features = initialize_trainning_df(auctions_18_20, "device_id")
auc_18_20_features.head(1)

""
ref_hash
1109595589636746168


> ### ***Feature 1: cantidad de veces que el usuario aparece en una subasta***

In [67]:
auctions_18_20['appearances_in_auctions'] = 1
auc_feature1 = auctions_18_20.groupby('device_id').agg({'appearances_in_auctions': 'count'})
auc_18_20_features = add_feature(auc_18_20_features, auc_feature1, "appearances_in_auctions")

In [68]:
auc_18_20_features.head(1)

,appearances_in_auctions
ref_hash,
1109595589636746168,128


> ### ***Feature 2: ¿apareció en una subasta el último día?***

In [69]:
auc_feature2 = auctions_18_20.loc[auctions_18_20["date"].dt.day == 20]
auc_feature2['user_appeared_last_day'] = 1
auc_feature2 = auc_feature2.groupby('device_id').agg({'user_appeared_last_day' : 'first'})
auc_18_20_features = add_feature(auc_18_20_features, auc_feature2, "user_appeared_last_day")

> ### ***Feature 3: en promedio, ¿ cuánto tardo el usuario en reaparecer en una subasta?***

In [70]:
def calculate_time_to_reappear(df, feature_date_name, feature_id_name):
    df_sorted = df.sort_values([feature_id_name, feature_date_name], ascending = True)
    df_sorted['time_to_reappear'] = np.\
    where(df_sorted[feature_id_name] == \
          df_sorted[feature_id_name].shift(), df_sorted[feature_date_name] - df_sorted[feature_date_name].shift(1), np.nan)
    df_sorted['time_to_reappear'] = df_sorted['time_to_reappear'].dt.total_seconds()
    df_sorted['time_to_reappear'] = df_sorted['time_to_reappear'].fillna(259200) #to check
    df_sorted = df_sorted.groupby(feature_id_name).agg({'time_to_reappear': 'mean'})
    return df_sorted

In [71]:
auc_feature3 = calculate_time_to_reappear(auctions_18_20, "date", "device_id")
auc_18_20_features = add_feature(auc_18_20_features, auc_feature3, "time_to_reappear")

> ### ***Feature 4: Día de la semana, día del mes, hora del día, segundo de la hora, minuto de la hora***

In [75]:
auc_feature4 = auctions_18_20.copy()

# Día de la semana
auc_feature4["dayofweek"] = auc_feature4.date.dt.dayofweek

# Día del mes
auc_feature4["day"] = auc_feature4.date.dt.day

# Hora del día
auc_feature4["hour"] = auc_feature4.date.dt.hour

# Segundo de la hora
auc_feature4["second"] = auc_feature4.date.dt.second

# Minuto de la hora
auc_feature4["minute"] = auc_feature4.date.dt.minute

In [82]:
auc_18_20_features = add_feature(auc_18_20_features, auc_feature4, "dayofweek")
auc_18_20_features = add_feature(auc_18_20_features, auc_feature4, "day")
auc_18_20_features = add_feature(auc_18_20_features, auc_feature4, "second")
auc_18_20_features = add_feature(auc_18_20_features, auc_feature4, "minute")

> ### ***Feature 5: Separación del día en 4 partes***

In [98]:
def divide_time_day(df):
    df.loc[(0 <= df["hour"]) & (df["hour"] < 6), 'timeofday'] = 0
    df.loc[(6 <= df["hour"]) & (df["hour"] < 13), 'timeofday'] = 1
    df.loc[(13 <= df["hour"]) & (df["hour"] < 19), 'timeofday'] = 2
    df.loc[(19 <= df["hour"]) & (df["hour"] < 24), 'timeofday'] = 3
    df["timeofday"] = df["timeofday"].astype(int)
    return df

In [99]:
auc_feature5 = auctions_18_20.copy()
auc_feature5["hour"] = auc_feature5.date.dt.hour
auc_feature5 = divide_time_day(auc_feature5)

In [101]:
auc_18_20_features = add_feature(auc_18_20_features, auc_feature5, "timeofday")

> ### ***Feature 6: Días feriados en Uruguay***

In [134]:
# En Uruguay hubo 3 días feriados en el rango de fecha que nos dieron
# El 18 y 19 de Abril fueron Jueves y Viernes Santo
# El 22 fue el día de Desembarco de los 33 orientales

def holidays_days(df):
    df.loc[(22 == df["day"]) | (19 == df["day"]) | (18 == df["day"]), 'holidaysday'] = 1
    return df

In [136]:
auc_feature6 = auctions_18_20.copy()
auc_feature6["day"] = auc_feature5.date.dt.day
auc_feature6 = holidays_days(auc_feature6)
auc_feature6 = auc_feature6.fillna(0)

In [137]:
auc_18_20_features = add_feature(auc_18_20_features, auc_feature6, "holidaysday")

> ### ***Feature 7: Días lluviosos en Uruguay***

In [138]:
# Días que llovieron del 18 al 26: El 26 fue el único día que llovió
def raining_days(df):
    df.loc[26 == df["day"], 'rainingdays'] = 1
    return df

In [142]:
auc_feature7 = auctions_18_20.copy()
auc_feature7["day"] = auc_feature7.date.dt.day
auc_feature7 = raining_days(auc_feature7)
auc_feature7 = auc_feature7.fillna(0)

In [144]:
auc_18_20_features = add_feature(auc_18_20_features, auc_feature7, "rainingdays")

> ### Save changes

In [120]:
save_changes(auc_18_20_features, "auc_18_20_features.csv")

# **Installs**

In [90]:
inst_18_20_features = initialize_trainning_df(installs_18_20, "ref_hash")
inst_18_20_features.head(1)

""
ref_hash
5230323462636548010


> ### ***Feature 1: cantidad de clicks realizados por el usuario***

In [ ]:
clicks_18_20['amount_of_clicks'] = 1
inst_feature1 = clicks_18_20.groupby('ref_hash').agg({'amount_of_clicks': 'count'})
inst_18_20_features = add_feature(inst_18_20_features, inst_feature1, "amount_of_clicks")

> ### ***Feature 2: ¿realizó una instalación?***

In [ ]:
installs_18_20['has_installed'] = 1
inst_feature2 = installs_18_20.groupby('ref_hash').agg({'has_installed': 'first'})
inst_18_20_features = add_feature(inst_18_20_features, inst_feature2, "has_installed")

> ### ***Feature 3: ¿realizó un click el último día?***

In [ ]:
inst_feature3 = clicks_18_20.loc[clicks_18_20["created"].dt.day == 20]
inst_feature3['user_clicked_last_day'] = 1
inst_feature3 = inst_feature3.groupby('ref_hash').agg({'user_clicked_last_day' : 'first'})
inst_18_20_features = add_feature(inst_18_20_features, inst_feature3, "user_clicked_last_day")

> ### ***Feature 4: ¿realizó una instalación el último día?***

In [ ]:
inst_feature4 = installs_18_20.loc[installs_18_20["created"].dt.day == 20]
inst_feature4['user_installed_last_day'] = 1
inst_feature4 = inst_feature4.groupby('ref_hash').agg({'user_installed_last_day' : 'first'})
inst_18_20_features = add_feature(inst_18_20_features, inst_feature4, "user_installed_last_day")

> ### ***Feature 5: cantidad de instalaciones realizadas***

In [ ]:
installs_18_20['amount_of_installs'] = 1
inst_feature5 = installs_18_20.groupby('ref_hash').agg({'amount_of_installs': 'count'})
inst_18_20_features = add_feature(inst_18_20_features, inst_feature5, "amount_of_installs")

> ### ***Feature 6: Mínimo de clicks en un día***

In [ ]:
clicks_18_20["day"] = clicks_18_20.created.dt.day
clicks_feature5 = clicks_18_20.groupby(["ref_hash", "day"]).agg({"cantidad":"sum"})
clicks_feature5 = pd.DataFrame(clicks_feature5.reset_index().groupby('ref_hash')['cantidad'].max()).reset_index()
clicks_feature5 = clicks_feature5.rename(columns={"cantidad":"cant_max_day"})

In [ ]:
clicks_18_20_features = add_feature(clicks_18_20_features, clicks_feature5, "cant_max_day")

> ### ***Feature 6: Máximo de clicks en un día***

In [156]:
clicks_feature6 = clicks_18_20.groupby(["ref_hash", "day"]).agg({"cantidad":"sum"})
clicks_feature6 = pd.DataFrame(clicks_feature5.reset_index().groupby('ref_hash')['cantidad'].min()).reset_index()
clicks_feature6 = clicks_feature5.rename(columns={"cantidad":"cant_min_day"})

NameError: name 'clicks_18_20' is not defined

In [157]:
clicks_18_20_features = add_feature(clicks_18_20_features, clicks_feature5, "cant_min_day")

NameError: name 'clicks_18_20_features' is not defined

> ### ***Feature 7: Promedio de tiempo entre clicks***

In [158]:
def calculate_time_to_reappear(df, feature_date_name, feature_id_name, function_type):
    df_sorted = df.sort_values([feature_id_name, feature_date_name], ascending = True)
    df_sorted['time_to_reappear'] = np.\
    where(df_sorted[feature_id_name] == \
          df_sorted[feature_id_name].shift(), df_sorted[feature_date_name] - df_sorted[feature_date_name].shift(1), np.nan)
    df_sorted['time_to_reappear'] = df_sorted['time_to_reappear'].dt.total_seconds()
    df_sorted['time_to_reappear'] = df_sorted['time_to_reappear'].fillna(259200) #to check
    df_sorted = df_sorted.groupby(feature_id_name).agg({'time_to_reappear': function_type})
    return df_sorted

In [ ]:
clicks_feature7 = calculate_time_to_reappear(clicks_18_20, "created", "ref_hash", "mean")
clicks_18_20_features = add_feature(clicks_18_20_features, clicks_feature5, "time_to_reappear")

> ### ***Feature 8: Tiempo máximo entre clicks***

In [ ]:
clicks_feature8 = calculate_time_to_reappear(clicks_18_20, "created", "ref_hash", "mean")

> ### ***Feature 9: Tiempo mínimo entre clicks***

In [ ]:
clicks_feature9 = calculate_time_to_reappear(clicks_18_20, "created", "ref_hash", "mean")

> ### Save changes

In [137]:
save_changes(inst_18_20_features, "inst_18_20_features.csv")